In [ ]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Install Unsloth (if not already installed)
!pip install git+https://github.com/huggingface/transformers
!pip install "unsloth [colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers "trl<0.9.0" peft accelerate bitsandbytes datasets
# !pip uninstall transformers
# !pip install git+https://github.com/huggingface/transformers


from datasets import load_dataset


# Import required libraries
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
import unsloth

# Clear unused CUDA memory
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

# Define model ID
model_id = "unsloth/Llama-3.2-11B-Vision-Instruct"
ds = load_dataset("rajistics/indian_food_images")


# Load model with Unsloth for optimized execution
# with unsloth.optimize():
model = MllamaForConditionalGeneration.from_pretrained(
       model_id, torch_dtype=torch.float16, device_map="auto"
)
model.gradient_checkpointing_enable()

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Download and load the image


ERROR: Operation cancelled by user
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-389e4hq4/unsloth_93223b989c7d43b2a9b4245279e91a17
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-389e4hq4/unsloth_93223b989c7d43b2a9b4245279e91a17


In [ ]:
!pip install pillow
from PIL import Image
import requests
url = "https://files.catbox.moe/rykgsb.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

# Prepare the prompt and inputs
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": "Detect and give output its food quality in terms of 0 to 1 and give it a boolean value of eatable.Give only in format : quality - 0 to 1 & boolean - true or false",
            },
        ],
    }
]

In [ ]:

input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, add_special_tokens=True, return_tensors="pt").to(
    model.device
)

# Generate the output
output = model.generate(**inputs, max_new_tokens=50)

# Decode and print the output
# print(processor.decode(output[0]))
decoded_output = processor.decode(output[0], skip_special_tokens=True)


In [ ]:
decoded_output

'user\n\nDetect and give output its food quality in terms of 0 to 1 and give it a boolean value of eatable.Give only in format : quality - 0 to 1 & boolean - true or falseassistant\n\nTo determine the food quality and whether it is eatable, we need to analyze the image provided. The image shows a hand holding a piece of bread with some filling inside. However, the image is blurry, making it difficult to identify the exact type'

In [ ]:
import re
quality_match = re.search(r"quality\s*-\s*(\d\.\d+)", decoded_output)
boolean_match = re.search(r"boolean\s*-\s*(true|false)", decoded_output, re.IGNORECASE)

# Final structured output
quality = float(quality_match.group(1)) if quality_match else None
eatable = boolean_match.group(1).lower() == "true" if boolean_match else None

print(f"Quality: {quality}, Eatable: {eatable}")

Quality: None, Eatable: True
